# 0 - collection des données


Au lieu d'utiliser l'interface de stackoverflow, il est possible de récuperer directement dans un dataframe avec des requetes python sur l'api afin de dépasser la limite de 50 000 rows, puis d'exporter pour traitement et modélisation.

In [5]:
import requests
import time
import pandas as pd

#resoudre captcha sur site avant et recup id sinon error 500 'https://data.stackexchange.com/stackoverflow/query/new'
def run_query_stackoverflow(sql_query, id_captcha):

    url = f'https://data.stackexchange.com/query/save/1/{id_captcha}'
    data = {"title": "P05_posts", "description": "", "sql" : sql_query, "g-recaptcha-response" : ""}

    response = requests.post(url, data)
    job_id = response.json().get('job_id')
    
    if(job_id is not None) :
        url = f'https://data.stackexchange.com/query/job/{job_id}'
        time.sleep(5)
        response = requests.get(url)
    
    data = response.json().get('resultSets')[0].get('rows')
    columns = [ col['name'] for col in response.json().get('resultSets')[0].get('columns')]
    
    df = pd.DataFrame(data , columns=columns)

    return df

In [6]:
periode = 300000
max_id = 5*periode+1
reqs = []
for i,j in zip(range(1,max_id,periode),range(periode,max_id,periode)):
    reqs.append(f"SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN {i} AND {j} AND ParentId IS NULL")

In [7]:
for req in reqs:
    print(req)

SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 1 AND 300000 AND ParentId IS NULL
SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 300001 AND 600000 AND ParentId IS NULL
SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 600001 AND 900000 AND ParentId IS NULL
SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 900001 AND 1200000 AND ParentId IS NULL
SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 1200001 AND 1500000 AND ParentId IS NULL


In [8]:
id_captcha = 1393531
responses = []
for req in reqs:
    responses.append(run_query_stackoverflow(req, id_captcha=id_captcha))

In [9]:
df_posts = pd.concat(responses)

In [10]:
df_posts.shape

(240600, 4)

In [12]:
df_posts.columns = ['id', 'body', 'title', 'tags']

In [13]:
df_posts

,id,body,title,tags
0,4,<p>I want to use a <code>Track-Bar</code> to c...,How to convert a Decimal to a Double in C#?,<c#><floating-point><type-conversion><double><...
1,6,<p>I have an absolutely positioned <code>div</...,Why did the width collapse in the percentage w...,<html><css><internet-explorer-7>
2,9,<p>Given a <code>DateTime</code> representing ...,How do I calculate someone's age based on a Da...,<c#><.net><datetime>
3,11,<p>Given a specific <code>DateTime</code> valu...,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...
4,13,<p>Is there a standard way for a web server to...,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...
...,...,...,...,...
49995,1417809,<p>Is there a way to trace/log sql from the co...,Debugging/tracing embedded sql,<c++><sql><oracle><trace><embedded-database>
49996,1417818,<p>I would like to use replace the value of a ...,HTML Entities with jQuery,<javascript><jquery><html><html-entities>
49997,1417819,<p>I got 3 <code>html.dropdownlists</code> out...,SelectedValue not binding for some of the Html...,<asp.net-mvc><binding><drop-down-menu><html-he...
49998,1417821,<p>I have a 2d array grid made as int GRID[10]...,Insert and remove rows from an int array in c++?,<visual-c++><native>


In [14]:
df_posts.to_csv("data/df_posts.csv", index=False)